In [12]:
fs = require('fs');
dsv = require('d3-dsv');
simpleStats = require('simple-statistics');
_ = require('lodash');

''

''

In [2]:
ct_census = dsv.csvParse(fs.readFileSync('too_big/ss14hct.csv', 'utf8'));
// from http://www2.census.gov/programs-surveys/acs/data/pums/2014/1-Year/csv_hct.zip
// let pums = dsv.csvParse(fs.readFileSync('too_big/usa_00002.csv', 'utf8'));
// let acs_14 = dsv.csvParse(fs.readFileSync('hh_14.csv', 'latin1'));

''

''

In [3]:
// these PUMA IDs represent the Bridgeport-Stamford-Norwalk, CT (Metro) according to geocorr
bridgeport_2014 = ct_census.filter(d => +d.PUMA >= 100 && +d.PUMA <= 105);

/*
"ADJINC
Adjustment factor for income and earnings dollar amounts(6 implied decimal places)
1008425 .2014 factor (1.008425)

Note: The value of ADJINC inflation-adjusts reported income to 2014 dollars.
ADJINC applies to variables FINCP and HINCP in the housing record, and variables
INTP, OIP, PAP, PERNP, PINCP, RETP, SEMP, SSIP, SSP, and WAGP in the person record. 

HINCP
Household income (past 12 months)
bbbbbbbbb .N/A(GQ/vacant)
000000000 .No household income
-00059999 .Loss of -$59,999 or more
-00000001..-00059998 .Loss of $1 to -$59,998
000000001 .$1 or Break even
000000002..999999999 .Total household income in dollars (Components are rounded)

Note: Use ADJINC to adjust HINCP to constant dollars."

https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMSDataDict14.txt
*/

bridgeport_2014 = bridgeport_2014.filter(d => d.HINCP !== ' ')
bridgeport_2014.forEach(d => d.HINCP == ' ' ? '' : d.ADJHH = (d.ADJINC/1000000)*d.HINCP);

''

''

In [4]:
function multiply (row, income_col, weight_col, puma_col, tar) {
    for (let i = 0; i < +row[weight_col]; i++) {
        tar.push(row[income_col]);
    }
}

function explode_weights (df, income_col, weight_col, puma_col) {
    /*
    Make a list of HHINCOME for calculating
    a quantile where each household's HHINCOME
    is repeated in the series n times, where
    n == weight_col
    */
    tar = []
    df.forEach(d => multiply(d,income_col,weight_col,puma_col,tar));
    return tar;
}

exploded_bridgeport = explode_weights(bridgeport_2014,"ADJHH","WGTP","PUMA");

''

''

In [5]:
simpleStats.quantile(exploded_bridgeport,0.95);

558969.9774999999

# Hallelujah!

In [6]:
acs_16 = dsv.csvParse(fs.readFileSync('hh_16.csv', 'latin1'));
mable = dsv.csvParse(fs.readFileSync('geocorr14_2014.csv', 'latin1'));

top_coded = acs_16.filter(d => !d.HD01_VD06)
bottom_coded = acs_16.filter(d => !d.HD01_VD02)

top_coded.forEach(d => d['States'] = d['GEO.display-label'].split(',')[1].split()[0].split('-'))

In [7]:
function make_stpuma(st, puma) {
    return st + ('00000'+puma).slice(-5);
}
make_stpuma(9,102)

'900102'

In [8]:
ct_census.forEach(d => d.stpuma = make_stpuma(d.ST,d.PUMA));

In [9]:
mable.stpuma = mable.forEach(d => d.stpuma = make_stpuma(d.state, d.puma12))
mable.slice(-5)

[ { state: '56',
    puma12: '00400',
    cbsa: '16220',
    stab: 'WY',
    cbsaname15: 'Casper, WY (Metro)',
    PUMAname: 'Natrona, Carbon & Converse Counties',
    pop14: '81624',
    afact: '0.732 ',
    stpuma: '5600400' },
  { state: '56',
    puma12: '00500',
    cbsa: ' ',
    stab: 'WY',
    cbsaname15: ' ',
    PUMAname: 'Sweetwater, Fremont, Uinta, Sublette & Hot Springs Counties--Wind River Reservation',
    pop14: '14873',
    afact: '0.122 ',
    stpuma: '5600500' },
  { state: '56',
    puma12: '00500',
    cbsa: '21740',
    stab: 'WY',
    cbsaname15: 'Evanston, WY (Micro)',
    PUMAname: 'Sweetwater, Fremont, Uinta, Sublette & Hot Springs Counties--Wind River Reservation',
    pop14: '20904',
    afact: '0.172 ',
    stpuma: '5600500' },
  { state: '56',
    puma12: '00500',
    cbsa: '40180',
    stab: 'WY',
    cbsaname15: 'Riverton, WY (Micro)',
    PUMAname: 'Sweetwater, Fremont, Uinta, Sublette & Hot Springs Counties--Wind River Reservation',
    pop14: '40703',

In [10]:
all_pumas = [];

function get_pumas(geo_id, geo_corr, rel) {
    cbsa = geo_corr.filter(d => +d.cbsa == +geo_id);
    if (cbsa.length == 0) {
        console.log(geo_id);
    }
    target = {};
    for (index in cbsa) {
        row = cbsa[index];
        
        target[+row.stpuma] = parseFloat(row.afact);
        rel.push(+row.stpuma);
    }
    return target;
}

top_coded.forEach(d => d.PUMAS = get_pumas(d['GEO.id2'], mable, all_pumas));

0640354


In [23]:
state_fips = _(mable).groupBy('stab').mapValues(d => _.maxBy(d,'state').state).value(); // [['state']].max()
state_fips['CA']

'06'

In [ ]:
/*
import zipfile,requests,io,os

def fetch_census(x,direct):
    zip_path = "https://www2.census.gov/programs-surveys/acs/data/pums/2016/1-Year/csv_h"+x+".zip"
    r = requests.get(zip_path, stream=True)
    zip_ref = zipfile.ZipFile(io.BytesIO(r.content))
    insert = ""
    if direct is not None:
        insert = direct+'/'
    dir_path = "too_big/"+insert+"2016/csv_h"+x+"/"
    zip_ref.extractall(dir_path)
    os.remove("too_big/"+insert+"2016/csv_h"+x+"/ACS2016_PUMS_README.pdf")

def download_unzip_census(ss_s,direct=None):
    ss_s.apply(lambda x: fetch_census(x.lower(),direct))
    
def make_path(year,state,direct):
    insert = ""
    if direct is not None:
        insert = direct +'/'
    return ("too_big/"+insert+str(year)+"/csv_h"+state+"/ss16h"+state+".csv")
    
def get_census_pums(ss_s,model,direct=None):
    tar = pandas.DataFrame(columns=model.columns)
    for index, s in ss_s.iteritems():
        print(s)
        file = make_path(2016,s.lower(),direct)
        new = pandas.read_csv(file) ## Grab for every topcoded state
        statefip = state_fips.loc[s.upper()]["state"]
        new["stpuma"] = new.apply(lambda x: make_stpuma(x["ST"],x["PUMA"]),axis=1)
        tar = pandas.concat([tar,new])
    return tar
*/

In [ ]:
pop_16 = dsv.csvParse(fs.readFileSync('ACS_16_1YR_B01003.csv', 'latin1'));
top_coded = top_coded.join(pop_16[["GEO.id2","HD01_VD01"]].set_index("GEO.id2"),on="GEO.id2")
top_coded["us_metro"] = top_coded.apply(lambda x: ("Metro" in x["GEO.display-label"]) & (" PR " not in x["GEO.display-label"]) & (x["HD01_VD01"] >= 500000),axis=1)
top_coded = top_coded[top_coded["us_metro"]==True]